In [22]:
from icecream import ic
import pandas as pd
from dateutil.relativedelta import relativedelta
# from dhs_preprocessing_functions import *
from pandarallel import pandarallel
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
# Initialization
pandarallel.initialize()

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [23]:
# folder: path = "/mnt/datadisk/data/surveys/DHS_final_raw_data/"
# work_dir = "/mnt/datadisk/data/Projects/water/pickles/"
input_dir = "/mnt/datadisk/data/Projects/water/inputs/"
# min_version = 3
overwrite_pqt = True
dataset_type = 'HR'  #, 'BR', 'CR', 'HW', 'IR', 'KR', 'MR', 'PR', 'AR'] ??['AN', 'FC, 'FP', 'HW', 'PV', 'SC'] < 25 surveys
group_by_col = 'adm2_gaul'

urban_rural_all_mode = 'U' # 'all', 'U', 'R'

in_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}_{urban_rural_all_mode}.pkl"

education_folder = f"/mnt/datadisk/data/preprocessed_data/DHS_n_more/"
education_adults_youth_f = f"{education_folder}additional_data/Education_Labels_Child_Youth_Aduld.csv"
education_f = f"{education_folder}additional_data/Education_Labels.csv"
sustainbench_f = f"{education_folder}additional_data/sustainbench_w_gaul.csv"
print(in_f)
# categorical_in_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}{add_str}.pkl"
fewsNet_f = f"{input_dir}fewsnet_gaul_adm2.csv"

out_f = f"{input_dir}5_grouped_df_V3_{dataset_type}_{group_by_col}_joined_with_ipc_{urban_rural_all_mode}.pkl"

/mnt/datadisk/data/Projects/water/inputs/grouped_df_V3_HR_adm2_gaul_U.pkl


In [24]:
df = pd.read_pickle(in_f)
df

Meta; adm1_gaul Meta; adm0_gaul  \
Meta; GEID_init Meta; adm2_gaul                                       
AOGE52FL        Benguela                   Benguela          Angola   
                Caala                        Huambo          Angola   
                Cabinda                     Cabinda          Angola   
                Cacuaco                      Luanda          Angola   
                Cacula                        Huila          Angola   
...                                             ...             ...   
ZWGE72FL        Mwenezi                    Masvingo        Zimbabwe   
                Nyanga                   Manicaland        Zimbabwe   
                Shurugwi                   Midlands        Zimbabwe   
                Umguza           Matabeleland North        Zimbabwe   
                Zvishavane                 Midlands        Zimbabwe   

                                 Meta; year  Meta; month Meta; date  \
Meta; GEID_init Meta; adm2_gaul                                       
AOGE52FL        Benguela               2006           11 2006-11-22   
                Caala                  2007            1 2007-01-19   
                Cabinda                2007            1 2007-01-07   
                Cacuaco                2006           12 2006-12-06   
                Cacula                 2006           12 2006-12-07   
...                                     ...          ...        ...   
ZWGE72FL        Mwenezi                2015            9 2015-09-09   
                Nyanga                 2015           11 2015-11-15   
                Shurugwi               2015            7 2015-07-23   
                Umguza                 2015            7 2015-07-07   
                Zvishavane             2015            7 2015-07-28   

                                 Meta one-hot encoding; adm0_gaul: Angola  \
Meta; GEID_init Meta; adm2_gaul                                             
AOGE52FL        Benguela                                              1.0   
                Caala                                                 1.0   
                Cabinda                                               1.0   
                Cacuaco                                               1.0   
                Cacula                                                1.0   
...                                                                   ...   
ZWGE72FL        Mwenezi                                               0.0   
                Nyanga                                                0.0   
                Shurugwi                                              0.0   
                Umguza                                                0.0   
                Zvishavane                                            0.0   

                                 Meta one-hot encoding; adm0_gaul: Benin  \
Meta; GEID_init Meta; adm2_gaul                                            
AOGE52FL        Benguela                                             0.0   
                Caala                                                0.0   
                Cabinda                                              0.0   
                Cacuaco                                              0.0   
                Cacula                                               0.0   
...                                                                  ...   
ZWGE72FL        Mwenezi                                              0.0   
                Nyanga                                               0.0   
                Shurugwi                                             0.0   
                Umguza                                               0.0   
                Zvishavane                                           0.0   

                                 Meta one-hot encoding; adm0_gaul: Burkina Faso  \
Meta; GEID_init Meta; adm2_gaul                                                   
AOGE52FL        Benguela       

In [25]:
food_security_df = pd.read_csv(fewsNet_f)
food_security_df.head()
ic(food_security_df.columns.to_list())
food_security_df = food_security_df[['adm2_name', 'adm1_name', 'adm0_name', 'month', 'year', 'CS', 'HA0']]
food_security_df['HA0'] = food_security_df['HA0'].fillna(0)
food_security_df['IPC + food help'] = food_security_df['CS'] + food_security_df['HA0']
food_security_df.drop(columns=['HA0'], inplace=True)
food_security_df.columns = ['adm2_name', 'adm1_name', 'adm0_name', 'month', 'year', 'IPC', 'IPC + food help']
print(len(food_security_df))
food_security_df.sample(10)

/tmp/ipykernel_1710663/3621970319.py:1: DtypeWarning: Columns (26,27,28,29,30,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  food_security_df = pd.read_csv(fewsNet_f)
ic| food_security_df.columns.to_list(): ['OBJECTID',
                                         'adm2_code',
                                         'adm2_name',
                                         'str2_year',
                                         'exp2_year',
                                         'adm1_code',
                                         'adm1_name',
                                         'status',
                                         'disp_area',
                                         'adm0_code',
                                         'adm0_name',
                                         'shape_leng',
                                         'FID_1',
                                         'Shape_Le_1',
                                   

104386


,adm2_name,adm1_name,adm0_name,month,year,IPC,IPC + food help
67317,Jimma,Oromia,Ethiopia,7,2013,1.0,1.0
18843,Opobo /Nkoro,Rivers,Nigeria,1,2013,1.0,1.0
33675,Kwaya Kusar,Borno,Nigeria,10,2015,2.0,2.0
11191,Kurmi,Taraba,Nigeria,7,2011,1.0,1.0
103826,Vangaindrano,Atsimo Atsinanana,Madagascar,10,2021,3.0,3.0
68955,Buhweju,Buhweju,Uganda,4,2014,1.0,1.0
52501,Balanga,Gombe,Nigeria,10,2020,1.0,1.0
69973,Al Mansura,Aden,Yemen,10,2014,2.0,2.0
22852,Okobo,Akwa Ibom,Nigeria,10,2013,1.0,1.0
32903,Orolu,Osun,Nigeria,10,2015,1.0,1.0


In [26]:
def aggregate_ipc(row, food_security_df, start_delta, end_delta, y_m_indicator):
    # Create datetime objects for the start and end dates
    in_date = row['Meta; date']
    if y_m_indicator == 'm':
        start_date = in_date + relativedelta(months=start_delta)
        end_date = in_date + relativedelta(months=end_delta)
    elif y_m_indicator == 'y':
        start_date = in_date + relativedelta(years=start_delta)
        end_date = in_date + relativedelta(years=end_delta)
    else:
        raise ValueError(f"y_m_indicator should be 'm' or 'y', not {y_m_indicator}")
    # Filter the data for the given time span and adm2_gaul
    # print(row.name)
    if group_by_col == 'adm2_gaul':
        adm2 = row.name[1]
    else:
        adm2 = row['Meta; adm2_gaul']
    
    df_time_span = food_security_df[(food_security_df['date'] >= start_date) & (food_security_df['date'] < end_date) & 
                                    (food_security_df['adm2_name'] == adm2) &
                                    (food_security_df['adm1_name'] == row['Meta; adm1_gaul']) &
                                    (food_security_df['adm0_name'] == row['Meta; adm0_gaul'])]


    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: mean'] = df_time_span['IPC'].mean()
    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: max'] = df_time_span['IPC'].max()
    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: min'] = df_time_span['IPC'].min()
    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: median'] = round(df_time_span['IPC'].median(), 0)

    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: mean'] = df_time_span['IPC + food help'].mean()
    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: max'] = df_time_span['IPC + food help'].max()
    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: min'] = df_time_span['IPC + food help'].min()
    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: median'] = round(df_time_span['IPC + food help'].median(), 0)

    return row

# Convert year and month to datetime
food_security_df['date'] = pd.to_datetime(food_security_df[['year', 'month']].assign(day=1))
print(food_security_df.shape)
print(df.shape)
print('Meta; date' in df.columns)

for start_delta, end_delta, m_y_ind in [(0, 5, 'm'), (6, 12, 'm'), (1, 2, 'y'), (2, 4, 'y'), (4, 6, 'y'), (6, 7, 'y'), (6, 10, 'y'), (10, 15, 'y'), (15, 20, 'y'), (20, 25, 'y'),
                                        (0, 1, 'y'), (0, 2, 'y'), (2, 6, 'y'), (6, 12, 'y'), (12, 20, 'y')]:
    df = df.parallel_apply(aggregate_ipc, axis=1, food_security_df=food_security_df, start_delta=start_delta, 
                                            end_delta=end_delta, y_m_indicator=m_y_ind)

df = df.reset_index()
df[df['FS; IPC + FH: 15-20y: mean'].notna()]

(104386, 8)
(7853, 1438)
True


,Meta; GEID_init,Meta; adm2_gaul,Meta; adm1_gaul,Meta; adm0_gaul,Meta; year,Meta; month,Meta; date,Meta one-hot encoding; adm0_gaul: Angola,Meta one-hot encoding; adm0_gaul: Benin,Meta one-hot encoding; adm0_gaul: Burkina Faso,...,FS; IPC + FH: 6-12y: min,FS; IPC + FH: 6-12y: median,FS; IPC: 12-20y: mean,FS; IPC: 12-20y: max,FS; IPC: 12-20y: min,FS; IPC: 12-20y: median,FS; IPC + FH: 12-20y: mean,FS; IPC + FH: 12-20y: max,FS; IPC + FH: 12-20y: min,FS; IPC + FH: 12-20y: median
118,BFGE32FL,Bale,Boucle Du Mouhoun,Burkina Faso,1999,2,1999-02-18,0.0,0.0,1.0,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.0,1.0,1.0
119,BFGE32FL,Bam,Centre-nord,Burkina Faso,1998,12,1998-12-07,0.0,0.0,1.0,...,1.0,2.0,1.076923,2.0,1.0,1.0,1.076923,2.0,1.0,1.0
120,BFGE32FL,Banwa,Boucle Du Mouhoun,Burkina Faso,1999,3,1999-03-02,0.0,0.0,1.0,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.0,1.0,1.0
121,BFGE32FL,Boulgou,Centre-est,Burkina Faso,1999,1,1999-01-08,0.0,0.0,1.0,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.0,1.0,1.0
122,BFGE32FL,Boulkiemde,Centre-ouest,Burkina Faso,1999,2,1999-02-23,0.0,0.0,1.0,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7779,ZWGE52FL,Shamva,Mashonaland Central,Zimbabwe,2006,2,2006-02-11,0.0,0.0,0.0,...,1.0,1.0,2.000000,3.0,1.0,2.0,2.100000,3.0,1.0,2.0
7780,ZWGE52FL,Shurugwi,Midlands,Zimbabwe,2006,1,2006-01-06,0.0,0.0,0.0,...,1.0,1.0,2.461538,3.0,1.0,3.0,2.461538,3.0,1.0,3.0
7781,ZWGE52FL,Umguza,Matabeleland North,Zimbabwe,2005,8,2005-08-04,0.0,0.0,0.0,...,1.0,1.0,2.357143,3.0,1.0,2.0,2.500000,3.0,1.0,3.0
7782,ZWGE52FL,Zvimba,Mashonaland West,Zimbabwe,2005,10,2005-10-24,0.0,0.0,0.0,...,1.0,1.0,1.538462,3.0,1.0,1.0,1.692308,3.0,1.0,1.0


In [27]:
education_df = pd.read_csv(education_f)
education_adults_youth_df = pd.read_csv(education_adults_youth_f)
education_df

,DHSID,HV007: year of interview,HV000: country code,Female Education PCA,Female Education PCA (min-max-scaled),EDI,EDI (min-max-scaled),EDI (UNESCO),EDI (UNESCO) (min-max-scaled),Imputed,...,LATNUM,LONGNUM,GEID,adm2_name,adm1_name,adm0_name,ISO3,TIF_name,MEAN_AGE,STD_AGE
0,MZ201500000001,2015,MZ,-3.083889,0.181055,0.275397,0.284240,NaN,NaN,True,...,-13.430448,40.510010,MZGE71FL,Mecufi,Cabo Delgado,Mozambique,MOZ,MZGE71FL00000001.tif,21.952381,21.531993
1,TZ201500000001,2015,TZ,0.534225,0.493104,0.467593,0.482607,NaN,NaN,False,...,-4.726384,35.912846,TZGE7AFL,Kondoa,Dodoma,Tanzania,TZA,TZGE7AFL00000001.tif,21.661538,21.644339
2,ZW201500000001,2015,ZW,1.888812,0.609932,0.612169,0.631826,0.578,0.722384,False,...,-17.303925,31.888711,ZWGE72FL,Uzumba-Maramba-Pfungwe,Mashonaland East,Zimbabwe,ZWE,ZWGE72FL00000001.tif,23.272727,19.734773
3,SN201500000001,2015,SN,0.429334,0.484057,0.503216,0.519375,0.514,0.629360,False,...,14.681623,-17.447857,SNGE7AFL,Dakar,Dakar,Senegal,SEN,SNGE7AFL00000001.tif,27.733333,19.123284
4,MW201500000002,2015,MW,-0.569319,0.397927,0.518254,0.534895,NaN,NaN,False,...,-12.855043,33.336077,MWGE7AFL,Kasungu,Central Region,Malawi,MWI,MWGE7AFL00000002.tif,22.388889,19.735972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,SN201900000059,2019,SN,-2.025488,0.272338,NaN,NaN,NaN,NaN,False,...,16.053994,-13.957100,SNGE8BFL,Podor,Saint louis,Senegal,SEN,SNGE8BFL00000059.tif,24.475610,21.848652
8042,SN201900000060,2019,SN,-3.542528,0.141499,NaN,NaN,NaN,NaN,False,...,16.414286,-15.253258,SNGE8BFL,Podor,Saint louis,Senegal,SEN,SNGE8BFL00000060.tif,22.757576,22.084496
8043,SN201900000070,2019,SN,3.150025,0.718707,NaN,NaN,0.282,0.292151,True,...,13.795034,-13.675060,SNGE8BFL,Tambacounda,Tambacounda,Senegal,SEN,SNGE8BFL00000070.tif,26.548387,18.451447
8044,SN201900000201,2019,SN,-0.341346,0.417589,NaN,NaN,NaN,NaN,False,...,12.692783,-15.583437,SNGE8BFL,Sedhiou,Sedhiou,Senegal,SEN,SNGE8BFL00000201.tif,23.534247,18.748394


In [28]:
education_adults_youth_df

,DHSYEAR,DHSID,URBAN_RURA,LATNUM,LONGNUM,GEID,adm2_name,adm1_name,adm0_name,principal component 1_child,HV007: year of interview,HV000: country code,HV001: cluster number,HV025: Type of place of residence,HV000.0: country name,principal component 1 (min-max-scaled)_child,principal component 1_aduld,principal component 1 (min-max-scaled)_aduld,principal component 1_youth,principal component 1 (min-max-scaled)_youth
0,2015.0,TZ201500000001,R,-4.726384,35.912846,TZGE7AFL,Kondoa,Dodoma,Tanzania,0.444082,2015,TZ,1,rural,Tanzania,0.516022,0.765668,0.442569,0.688759,0.507157
1,2015.0,ZW201500000001,R,-17.303925,31.888711,ZWGE72FL,Uzumba-Maramba-Pfungwe,Mashonaland East,Zimbabwe,-0.781098,2015,ZW,1,rural,Zimbabwe,0.370218,1.569454,0.556490,0.831399,0.532511
2,2015.0,SN201500000001,U,14.681623,-17.447857,SNGE7AFL,Dakar,Dakar,Senegal,-1.691298,2015,SN,1,urban,Senegal,0.261898,-0.810731,0.219146,-0.119323,0.363524
3,2015.0,MW201500000002,R,-12.855043,33.336077,MWGE7AFL,Kasungu,Central Region,Malawi,-0.402957,2015,MW,2,rural,Malawi,0.415219,-0.431399,0.272909,-0.727331,0.255454
4,2015.0,TZ201500000002,R,-4.364093,35.852918,TZGE7AFL,Kondoa,Dodoma,Tanzania,-0.262145,2015,TZ,2,rural,Tanzania,0.431977,1.409354,0.533799,0.054487,0.394418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7457,2019.0,SN201900000002,U,14.711251,-17.448647,SNGE8BFL,Dakar,Dakar,Senegal,-2.602198,2019,SN,2,urban,Senegal,0.153496,0.825012,0.450980,1.744360,0.694786
7458,2019.0,SN201900000054,R,16.477810,-15.649649,SNGE8BFL,Dagana,Saint louis,Senegal,2.231289,2019,SN,54,rural,Senegal,0.728711,-2.131972,0.031886,-1.582194,0.103506
7459,2019.0,SN201900000059,R,16.053994,-13.957100,SNGE8BFL,Podor,Saint louis,Senegal,1.024810,2019,SN,59,rural,Senegal,0.585132,-1.595985,0.107851,-0.684831,0.263008
7460,2019.0,SN201900000060,R,16.414286,-15.253258,SNGE8BFL,Podor,Saint louis,Senegal,1.821255,2019,SN,60,rural,Senegal,0.679914,-2.356948,0.000000,-1.777370,0.068814


In [29]:
rest_cols = [c for c in education_adults_youth_df.columns if c not in education_df.columns] + ['DHSID']
# merge education data
edu_df = pd.merge(education_df, education_adults_youth_df[rest_cols], on=['DHSID'], how='left')
print(list(edu_df.columns))
drop_cols = ['DHSID', 'HV000: country code', 'households', 'URBAN_RURA', 'LATNUM', 'LONGNUM', 'GEID', 'ISO3', 'TIF_name', 'MEAN_AGE', 'STD_AGE', 'HV001: cluster number', 'HV025: Type of place of residence', 'HV000.0: country name']
edu_df.drop(columns=[c for c in drop_cols if c in edu_df.columns], inplace=True)
for col in edu_df.columns:
    print(col, len(edu_df[col].dropna())/len(edu_df) *100)
    print(edu_df[col].value_counts(dropna=False, normalize=True))

['DHSID', 'HV007: year of interview', 'HV000: country code', 'Female Education PCA', 'Female Education PCA (min-max-scaled)', 'EDI', 'EDI (min-max-scaled)', 'EDI (UNESCO)', 'EDI (UNESCO) (min-max-scaled)', 'Imputed', 'households', 'DHSYEAR', 'URBAN_RURA', 'LATNUM', 'LONGNUM', 'GEID', 'adm2_name', 'adm1_name', 'adm0_name', 'ISO3', 'TIF_name', 'MEAN_AGE', 'STD_AGE', 'principal component 1_child', 'HV001: cluster number', 'HV025: Type of place of residence', 'HV000.0: country name', 'principal component 1 (min-max-scaled)_child', 'principal component 1_aduld', 'principal component 1 (min-max-scaled)_aduld', 'principal component 1_youth', 'principal component 1 (min-max-scaled)_youth']
HV007: year of interview 100.0
HV007: year of interview
2018    0.402187
2015    0.281879
2019    0.146160
2017    0.085384
2016    0.084390
Name: proportion, dtype: float64
Female Education PCA 100.0
Female Education PCA
-5.137032    0.009197
-5.089328    0.000373
-5.005625    0.000249
-4.837268    0.000249

In [30]:
edu_df = edu_df[edu_df['Imputed'] == False]
edu_df.drop(columns=['Imputed'], inplace=True)
edu_df[edu_df["DHSYEAR"] != edu_df["HV007: year of interview"]]

,HV007: year of interview,Female Education PCA,Female Education PCA (min-max-scaled),EDI,EDI (min-max-scaled),EDI (UNESCO),EDI (UNESCO) (min-max-scaled),DHSYEAR,adm2_name,adm1_name,adm0_name,principal component 1_child,principal component 1 (min-max-scaled)_child,principal component 1_aduld,principal component 1 (min-max-scaled)_aduld,principal component 1_youth,principal component 1 (min-max-scaled)_youth


In [31]:
### Aggregating the data: (categorical values are aggregated further below)

# Define the columns to include in each type of aggregation
group_by_cols = ['adm0_name', 'adm1_name', 'adm2_name', 'DHSYEAR']
cols_for_stats = [col for col in edu_df.columns if col not in group_by_cols and col != 'HV007: year of interview']

# Group the DataFrame
grouped = edu_df.groupby(group_by_cols)

# Perform the aggregations
agg_df = grouped[cols_for_stats].agg('mean')

# mean_df = grouped[cols_for_mean].agg('mean')
first_df = grouped[group_by_cols + ['HV007: year of interview']].agg('first')

# Combine the results into a single DataFrame
edu_agg_df = pd.concat([first_df, agg_df], axis=1).reset_index(drop=True)

edu_agg_df

,adm0_name,adm1_name,adm2_name,DHSYEAR,HV007: year of interview,Female Education PCA,Female Education PCA (min-max-scaled),EDI,EDI (min-max-scaled),EDI (UNESCO),EDI (UNESCO) (min-max-scaled),principal component 1_child,principal component 1 (min-max-scaled)_child,principal component 1_aduld,principal component 1 (min-max-scaled)_aduld,principal component 1_youth,principal component 1 (min-max-scaled)_youth
0,Angola,Bengo,Ambriz,2015.0,2015,-0.794291,0.378524,0.439209,0.453312,NaN,NaN,-0.310652,0.426204,-0.422030,0.274236,-1.193443,0.172604
1,Angola,Bengo,Bula Atumba,2015.0,2015,-3.340671,0.158909,0.172222,0.177752,NaN,NaN,2.085018,0.711304,-1.829539,0.074750,-1.810497,0.062926
2,Angola,Bengo,Dande,2015.0,2015,1.080300,0.540201,0.596036,0.615175,NaN,NaN,-1.132649,0.328381,0.841928,0.453378,0.023338,0.388882
3,Angola,Bengo,Dembos,2015.0,2015,-1.230428,0.340909,0.464002,0.478901,NaN,NaN,0.043287,0.468325,-0.866186,0.211286,-1.096704,0.189799
4,Angola,Bengo,Icolo E Bengo,2015.0,2015,-4.065513,0.096394,0.126389,0.130447,NaN,NaN,2.856312,0.803092,-2.028548,0.046544,-2.164520,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,Zimbabwe,Midlands,Gweru,2015.0,2015,2.636273,0.674398,0.672066,0.693646,0.586,0.734012,-1.122044,0.329643,2.339653,0.665650,0.996906,0.561929
1622,Zimbabwe,Midlands,Kwekwe,2015.0,2015,2.665429,0.676912,0.641149,0.661736,0.586,0.734012,-0.949041,0.350232,2.384278,0.671975,1.234390,0.604141
1623,Zimbabwe,Midlands,Mberengwa,2015.0,2015,1.663043,0.590460,0.580272,0.598905,0.586,0.734012,-0.711294,0.378525,1.687652,0.573242,0.414561,0.458420
1624,Zimbabwe,Midlands,Shurugwi,2015.0,2015,2.132703,0.630967,0.633195,0.653527,0.586,0.734012,-0.779511,0.370407,2.272620,0.656150,0.483507,0.470675


In [32]:
for col in edu_agg_df.select_dtypes(include=[np.number]).columns:
    print(col)
    print(len(edu_agg_df[col].dropna())/len(edu_agg_df) *100, len(edu_df[col].dropna())/len(edu_df) *100)
    print(edu_agg_df[col].mean(), edu_df[col].mean())
    print(edu_agg_df[col].median(), edu_df[col].median())
    print(edu_agg_df[col].std(), edu_df[col].std())
    print('\n')



DHSYEAR
100.0 100.0
2017.1531365313654 2017.1114982578397
2018.0 2018.0
1.3475605318078208 1.4594199673606156


HV007: year of interview
100.0 100.0
2017.1531365313654 2017.1114982578397
2018.0 2018.0
1.3475605318078208 1.4594199673606156


Female Education PCA
100.0 100.0
-0.19912141242913528 -0.2997914907593215
-0.35173371450775476 -0.34917379286826916
2.5280366322874426 2.5236248485332125


Female Education PCA (min-max-scaled)
100.0 100.0
0.42985554934886777 0.4211731332632381
0.41669331158755013 0.41691409521252876
0.21803366288483328 0.2176531631090369


EDI
96.37146371463714 93.00455641919056
0.42959289742289586 0.4339313713796106
0.45431134259259254 0.45461691086691086
0.19337729515768232 0.19813545294642365


EDI (min-max-scaled)
96.37146371463714 93.00455641919056
0.4433871647713375 0.4478649475248274
0.46889932148318036 0.4692147015828208
0.19958665784623172 0.20449760051809782


EDI (UNESCO)
63.40713407134071 54.409005628517825
0.48507177497575166 0.4608206896551724
0.506 0

In [33]:
# Merge both datasets
df2 = pd.merge(df, edu_agg_df, left_on=['Meta; adm0_gaul', 'Meta; adm1_gaul', 'Meta; adm2_gaul', 'Meta; year'], right_on=['adm0_name', 'adm1_name', 'adm2_name', 'DHSYEAR'], how='left')
drop_cols = ['adm0_name', 'adm1_name', 'adm2_name', 'DHSYEAR', 'HV007: year of interview']
df2.drop(columns=[c for c in drop_cols if c in df2.columns], inplace=True)
replace_d = {c: 'DHS Education; ' + c for c in edu_agg_df.columns if c not in drop_cols}
print(replace_d)
df2.rename(columns=replace_d, inplace=True)
df2

{'Female Education PCA': 'DHS Education; Female Education PCA', 'Female Education PCA (min-max-scaled)': 'DHS Education; Female Education PCA (min-max-scaled)', 'EDI': 'DHS Education; EDI', 'EDI (min-max-scaled)': 'DHS Education; EDI (min-max-scaled)', 'EDI (UNESCO)': 'DHS Education; EDI (UNESCO)', 'EDI (UNESCO) (min-max-scaled)': 'DHS Education; EDI (UNESCO) (min-max-scaled)', 'principal component 1_child': 'DHS Education; principal component 1_child', 'principal component 1 (min-max-scaled)_child': 'DHS Education; principal component 1 (min-max-scaled)_child', 'principal component 1_aduld': 'DHS Education; principal component 1_aduld', 'principal component 1 (min-max-scaled)_aduld': 'DHS Education; principal component 1 (min-max-scaled)_aduld', 'principal component 1_youth': 'DHS Education; principal component 1_youth', 'principal component 1 (min-max-scaled)_youth': 'DHS Education; principal component 1 (min-max-scaled)_youth'}


,Meta; GEID_init,Meta; adm2_gaul,Meta; adm1_gaul,Meta; adm0_gaul,Meta; year,Meta; month,Meta; date,Meta one-hot encoding; adm0_gaul: Angola,Meta one-hot encoding; adm0_gaul: Benin,Meta one-hot encoding; adm0_gaul: Burkina Faso,...,DHS Education; EDI,DHS Education; EDI (min-max-scaled),DHS Education; EDI (UNESCO),DHS Education; EDI (UNESCO) (min-max-scaled),DHS Education; principal component 1_child,DHS Education; principal component 1 (min-max-scaled)_child,DHS Education; principal component 1_aduld,DHS Education; principal component 1 (min-max-scaled)_aduld,DHS Education; principal component 1_youth,DHS Education; principal component 1 (min-max-scaled)_youth
0,AOGE52FL,Benguela,Benguela,Angola,2006,11,2006-11-22,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AOGE52FL,Caala,Huambo,Angola,2007,1,2007-01-19,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AOGE52FL,Cabinda,Cabinda,Angola,2007,1,2007-01-07,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AOGE52FL,Cacuaco,Luanda,Angola,2006,12,2006-12-06,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AOGE52FL,Cacula,Huila,Angola,2006,12,2006-12-07,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7848,ZWGE72FL,Mwenezi,Masvingo,Zimbabwe,2015,9,2015-09-09,0.0,0.0,0.0,...,0.468802,0.483855,0.588,0.736919,0.729570,0.549997,1.334320,0.523164,0.100486,0.402594
7849,ZWGE72FL,Nyanga,Manicaland,Zimbabwe,2015,11,2015-11-15,0.0,0.0,0.0,...,0.588724,0.607628,0.562,0.699128,-0.865433,0.360181,1.686166,0.573032,0.930580,0.550140
7850,ZWGE72FL,Shurugwi,Midlands,Zimbabwe,2015,7,2015-07-23,0.0,0.0,0.0,...,0.633195,0.653527,0.586,0.734012,-0.779511,0.370407,2.272620,0.656150,0.483507,0.470675
7851,ZWGE72FL,Umguza,Matabeleland North,Zimbabwe,2015,7,2015-07-07,0.0,0.0,0.0,...,0.629854,0.650079,NaN,NaN,-1.257618,0.313509,2.186555,0.643952,1.124329,0.584578


In [34]:
df = df2
sustainbench_df = pd.read_csv(sustainbench_f)
sustainbench_df = sustainbench_df[(sustainbench_df['adm0_name'].notna())]
print(sustainbench_df.columns)
vc = sustainbench_df[["adm0_name", "year"]].value_counts(normalize=True, dropna=False)
vc

Index(['DHSID_EA', 'cname', 'year', 'lat', 'lon', 'n_asset', 'asset_index',
       'n_water', 'water_index', 'n_sanitation', 'sanitation_index',
       'under5_mort', 'n_under5_mort', 'women_edu', 'women_bmi', 'n_women_edu',
       'n_women_bmi', 'cluster_id', 'adm1fips', 'adm1dhs', 'urban',
       'adm2_name', 'adm1_name', 'adm0_name'],
      dtype='object')


/tmp/ipykernel_1710663/1568250208.py:2: DtypeWarning: Columns (21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  sustainbench_df = pd.read_csv(sustainbench_f)


adm0_name             year
Egypt                 2014    0.029402
Kenya                 2014    0.027079
Nigeria               2018    0.023576
Egypt                 2005    0.021526
                      2008    0.020382
                                ...   
Syrian Arab Republic  2007    0.000017
Guinea                2013    0.000017
Gambia                2017    0.000017
                      2010    0.000017
Sudan                 2005    0.000017
Name: proportion, Length: 194, dtype: float64

In [35]:
### Aggregating the data: (categorical values are aggregated further below)

# Define the columns to include in each type of aggregation
group_by_cols = ['adm0_name', 'adm1_name', 'adm2_name', 'year']

cols = [('n_asset', 'asset_index'), ('n_water', 'water_index'), ('n_sanitation', 'sanitation_index'), 
        ('n_under5_mort', 'under5_mort'), ('n_women_edu', 'women_edu'), ('n_women_bmi', 'women_bmi')]

# Define a function to calculate weighted mean
def weighted_mean(data, weights):
    return np.average(data, weights=weights)

# Calculate weighted averages for each group
grouped = sustainbench_df.groupby(group_by_cols).apply(
    lambda x: pd.Series({
        'asset_index': weighted_mean(x['asset_index'], x['n_asset']),
        'water_index': weighted_mean(x['water_index'], x['n_water']),
        'sanitation_index': weighted_mean(x['sanitation_index'], x['n_sanitation']),
        'under5_mort': weighted_mean(x['under5_mort'], x['n_under5_mort']),
        'women_edu': weighted_mean(x['women_edu'], x['n_women_edu']),
        'women_bmi': weighted_mean(x['women_bmi'], x['n_women_bmi']),
        
        # Add similar lines for other columns as needed
    })
)

sustainbench_agg_df = grouped.reset_index()

grouped

asset_index  water_index  \
adm0_name adm1_name adm2_name   year                             
Angola    Bengo     Ambriz      2015    -0.254899     3.865385   
                    Bula Atumba 2015    -3.566753     1.000000   
                    Dande       2006          NaN          NaN   
                                2011    -0.057411     3.055556   
                                2015    -0.052050     2.686684   
...                                           ...          ...   
Zimbabwe  Midlands  Shurugwi    2015     1.042259     3.901235   
                    Zvishavane  1999          NaN          NaN   
                                2005     0.180565     3.328947   
                                2010     0.563056     3.531646   
                                2015     1.461591     4.024390   

                                      sanitation_index  under5_mort  \
adm0_name adm1_name adm2_name   year                                  
Angola    Bengo     Ambriz      2015          2.653846    32.258065   
                    Bula Atumba 2015          1.078431     0.000000   
                    Dande       2006               NaN     0.000000   
                                2011          3.069444    12.820513   
                                2015          3.180157     6.269592   
...                                                ...          ...   
Zimbabwe  Midlands  Shurugwi    2015          3.629630    38.461538   
                    Zvishavane  1999               NaN    45.454545   
                                2005          3.315789    18.518519   
                                2010          3.620253     0.000000   
                                2015          4.085366     0.000000   

                                      women_edu  women_bmi  
adm0_name adm1_name adm2_name   year                        
Angola    Bengo     Ambriz      2015   5.435897        NaN  
                    Bula Atumba 2015   2.800000        NaN  
                    Dande       2006   4.080000        NaN  
                                2011   5.666667        NaN  
                                2015   5.600000        NaN  
...                                         ...        ...  
Zimbabwe  Midlands  Shurugwi    2015   9.662338  24.858358  
                    Zvishavane  1999   8.612245  23.338000  
                                2005   7.923077  22.847027  
                                2010  10.000000  22.463117  
                                2015   9.800000  24.965224  

[13494 rows x 6 columns]

In [36]:
for col in sustainbench_agg_df.select_dtypes(include=[np.number]).columns:
    print(col)
    print(len(sustainbench_agg_df[col].dropna())/len(sustainbench_agg_df) *100, len(sustainbench_df[col].dropna())/len(sustainbench_df) *100)
    print(sustainbench_agg_df[col].mean(), sustainbench_df[col].mean())
    print(sustainbench_agg_df[col].median(), sustainbench_df[col].median())
    print(sustainbench_agg_df[col].std(), sustainbench_df[col].std())
    print('\n')

year
100.0 100.0
2010.0972283978065 2010.180393965797
2011.0 2011.0
5.839489664528806 5.747072531847052


asset_index
73.21031569586482 69.70426938649993
-0.7819299512841685 -0.4901913092491207
-1.0247614940383964 -0.824184931834419
1.4653337293156228 1.7529223222943557


water_index
73.23254779902179 69.70939470042539
3.27434339245057 3.4797178690224135
3.341186252771617 3.64
1.020972533200376 1.2059824859742476


sanitation_index
73.23254779902179 69.70939470042539
2.4902531110042703 2.6611271866526427
2.39285714285714 2.55
0.9399256807469091 1.078374428976398


under5_mort
89.09885875203794 95.54268532280935
19.166602221475994 17.400914541724948
0.0 0.0
29.681411851612733 34.99556059501546


women_edu
98.82910923373352 99.58143269608597
5.210178989625497 5.415941457723851
5.1 5.276228906823185
3.161309375370038 3.5108783908307752


women_bmi
60.78257003112495 75.61033946662566
23.43325392066449 23.798727306473115
22.673043154761885 22.99625
3.0386387443209415 3.3759663519328016




In [37]:
print("Meta; adm2_gaul" in df.columns)
df

True


,Meta; GEID_init,Meta; adm2_gaul,Meta; adm1_gaul,Meta; adm0_gaul,Meta; year,Meta; month,Meta; date,Meta one-hot encoding; adm0_gaul: Angola,Meta one-hot encoding; adm0_gaul: Benin,Meta one-hot encoding; adm0_gaul: Burkina Faso,...,DHS Education; EDI,DHS Education; EDI (min-max-scaled),DHS Education; EDI (UNESCO),DHS Education; EDI (UNESCO) (min-max-scaled),DHS Education; principal component 1_child,DHS Education; principal component 1 (min-max-scaled)_child,DHS Education; principal component 1_aduld,DHS Education; principal component 1 (min-max-scaled)_aduld,DHS Education; principal component 1_youth,DHS Education; principal component 1 (min-max-scaled)_youth
0,AOGE52FL,Benguela,Benguela,Angola,2006,11,2006-11-22,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AOGE52FL,Caala,Huambo,Angola,2007,1,2007-01-19,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AOGE52FL,Cabinda,Cabinda,Angola,2007,1,2007-01-07,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AOGE52FL,Cacuaco,Luanda,Angola,2006,12,2006-12-06,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AOGE52FL,Cacula,Huila,Angola,2006,12,2006-12-07,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7848,ZWGE72FL,Mwenezi,Masvingo,Zimbabwe,2015,9,2015-09-09,0.0,0.0,0.0,...,0.468802,0.483855,0.588,0.736919,0.729570,0.549997,1.334320,0.523164,0.100486,0.402594
7849,ZWGE72FL,Nyanga,Manicaland,Zimbabwe,2015,11,2015-11-15,0.0,0.0,0.0,...,0.588724,0.607628,0.562,0.699128,-0.865433,0.360181,1.686166,0.573032,0.930580,0.550140
7850,ZWGE72FL,Shurugwi,Midlands,Zimbabwe,2015,7,2015-07-23,0.0,0.0,0.0,...,0.633195,0.653527,0.586,0.734012,-0.779511,0.370407,2.272620,0.656150,0.483507,0.470675
7851,ZWGE72FL,Umguza,Matabeleland North,Zimbabwe,2015,7,2015-07-07,0.0,0.0,0.0,...,0.629854,0.650079,NaN,NaN,-1.257618,0.313509,2.186555,0.643952,1.124329,0.584578


In [38]:
# Merge both datasets
df2 = pd.merge(df, sustainbench_agg_df, left_on=['Meta; adm0_gaul', 'Meta; adm1_gaul', 'Meta; adm2_gaul', 'Meta; year'], right_on=['adm0_name', 'adm1_name', 'adm2_name', 'year'], how='left')
replace_d = {c: 'DHS Sustainbench; ' + c for c in sustainbench_agg_df.columns if c not in ['adm0_name', 'adm1_name', 'adm2_name', 'year']}
df2.drop(columns=['adm0_name', 'adm1_name', 'adm2_name', 'year'], inplace=True)
df2.rename(columns=replace_d, inplace=True)
for col in df2.columns:
    if not 'region' in col and not 'language' in col:
        print(col)

Meta; GEID_init
Meta; adm2_gaul
Meta; adm1_gaul
Meta; adm0_gaul
Meta; year
Meta; month
Meta; date
Meta one-hot encoding; adm0_gaul: Angola
Meta one-hot encoding; adm0_gaul: Benin
Meta one-hot encoding; adm0_gaul: Burkina Faso
Meta one-hot encoding; adm0_gaul: Burundi
Meta one-hot encoding; adm0_gaul: Cameroon
Meta one-hot encoding; adm0_gaul: Central African Republic
Meta one-hot encoding; adm0_gaul: Chad
Meta one-hot encoding; adm0_gaul: Comoros
Meta one-hot encoding; adm0_gaul: Côte d'Ivoire
Meta one-hot encoding; adm0_gaul: Democratic Republic of the Congo
Meta one-hot encoding; adm0_gaul: Egypt
Meta one-hot encoding; adm0_gaul: Ethiopia
Meta one-hot encoding; adm0_gaul: Gabon
Meta one-hot encoding; adm0_gaul: Gambia
Meta one-hot encoding; adm0_gaul: Ghana
Meta one-hot encoding; adm0_gaul: Guinea
Meta one-hot encoding; adm0_gaul: Kenya
Meta one-hot encoding; adm0_gaul: Lesotho
Meta one-hot encoding; adm0_gaul: Liberia
Meta one-hot encoding; adm0_gaul: Madagascar
Meta one-hot encodin

In [39]:
df = df2
# save the dataframe
df.to_pickle(out_f)
df.to_csv(out_f.replace('.pkl', '.csv'), index=False)

In [40]:
df

,Meta; GEID_init,Meta; adm2_gaul,Meta; adm1_gaul,Meta; adm0_gaul,Meta; year,Meta; month,Meta; date,Meta one-hot encoding; adm0_gaul: Angola,Meta one-hot encoding; adm0_gaul: Benin,Meta one-hot encoding; adm0_gaul: Burkina Faso,...,DHS Education; principal component 1_aduld,DHS Education; principal component 1 (min-max-scaled)_aduld,DHS Education; principal component 1_youth,DHS Education; principal component 1 (min-max-scaled)_youth,DHS Sustainbench; asset_index,DHS Sustainbench; water_index,DHS Sustainbench; sanitation_index,DHS Sustainbench; under5_mort,DHS Sustainbench; women_edu,DHS Sustainbench; women_bmi
0,AOGE52FL,Benguela,Benguela,Angola,2006,11,2006-11-22,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,9.606061,NaN
1,AOGE52FL,Caala,Huambo,Angola,2007,1,2007-01-19,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AOGE52FL,Cabinda,Cabinda,Angola,2007,1,2007-01-07,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AOGE52FL,Cacuaco,Luanda,Angola,2006,12,2006-12-06,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,6.853659,NaN
4,AOGE52FL,Cacula,Huila,Angola,2006,12,2006-12-07,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.247788,4.303922,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7848,ZWGE72FL,Mwenezi,Masvingo,Zimbabwe,2015,9,2015-09-09,0.0,0.0,0.0,...,1.334320,0.523164,0.100486,0.402594,-1.429001,2.561905,2.171429,9.803922,7.857143,24.167841
7849,ZWGE72FL,Nyanga,Manicaland,Zimbabwe,2015,11,2015-11-15,0.0,0.0,0.0,...,1.686166,0.573032,0.930580,0.550140,-0.024322,3.417476,3.097087,32.786885,9.022222,24.186098
7850,ZWGE72FL,Shurugwi,Midlands,Zimbabwe,2015,7,2015-07-23,0.0,0.0,0.0,...,2.272620,0.656150,0.483507,0.470675,1.042259,3.901235,3.629630,38.461538,9.662338,24.858358
7851,ZWGE72FL,Umguza,Matabeleland North,Zimbabwe,2015,7,2015-07-07,0.0,0.0,0.0,...,2.186555,0.643952,1.124329,0.584578,0.136341,3.769231,2.951923,0.000000,9.517647,23.684026


In [41]:
for col in df.columns:
    if not 'region' in col and not 'language' in col and not 'encoding' in col:
        print(col, len(df[col].dropna())/len(df) *100)

Meta; GEID_init 100.0
Meta; adm2_gaul 100.0
Meta; adm1_gaul 100.0
Meta; adm0_gaul 100.0
Meta; year 100.0
Meta; month 100.0
Meta; date 100.0
DHS Num; number of household members: mean 100.0
DHS Num; number of household members: median 100.0
DHS Num; number of household members: std 100.0
DHS Num; number of household members: skewness 100.0
DHS Num; number of household members: kurtosis 100.0
DHS Num; owns sheep: mean 70.0114605883102
DHS Num; owns sheep: median 70.0114605883102
DHS Num; owns sheep: std 70.0114605883102
DHS Num; owns sheep: skewness 70.0114605883102
DHS Num; owns sheep: kurtosis 70.0114605883102
DHS Num; number of eligible children for height and weight: mean 57.59582325226028
DHS Num; number of eligible children for height and weight: median 57.59582325226028
DHS Num; number of eligible children for height and weight: std 57.59582325226028
DHS Num; number of eligible children for height and weight: skewness 57.59582325226028
DHS Num; number of eligible children for heig

In [42]:
df[df['Meta; GEID_init'].isna()]

,Meta; GEID_init,Meta; adm2_gaul,Meta; adm1_gaul,Meta; adm0_gaul,Meta; year,Meta; month,Meta; date,Meta one-hot encoding; adm0_gaul: Angola,Meta one-hot encoding; adm0_gaul: Benin,Meta one-hot encoding; adm0_gaul: Burkina Faso,...,DHS Education; principal component 1_aduld,DHS Education; principal component 1 (min-max-scaled)_aduld,DHS Education; principal component 1_youth,DHS Education; principal component 1 (min-max-scaled)_youth,DHS Sustainbench; asset_index,DHS Sustainbench; water_index,DHS Sustainbench; sanitation_index,DHS Sustainbench; under5_mort,DHS Sustainbench; women_edu,DHS Sustainbench; women_bmi
